## Predicting a Biological Response.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/from-coursera"))

# Any results you write to the current directory are saved as output.

This is my first notebook and first English article. I apologize to native English speakers. This is my solution in the course on Cursera. Hope it might be useful here.

In [ ]:
import math
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split

You must Load a selection from the game-data file.csv using pandas and convert it to a numpy array (the parameter values at dataframe). In the first column of the data file it is written whether there was a reaction or not. All other columns (d1 - d1776) contain a variety of molecular characteristics such as size, shape, etc. Split the sample into training and test using the train_test_split function with parameters test_size = 0.8 and random_state = 241.

In [ ]:
df = pd.read_csv('../input/from-coursera/gbm-data.csv')
df.tail()

In [ ]:
df_arr = df.values

In [ ]:
y = df_arr[:,0]

In [ ]:
X = df_arr[:,1:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=241)

Train the Gradient Boosting Classifier with the parameters n_estimators=250, verbose=True, random_state=241 and for each value learning_rate from the list [1, 0.5, 0.3, 0.2, 0.1] do the following:

* Use the staged_decision_function method to predict quality on the training and test samples at each iteration.
* Convert the resulting prediction using the sigmoid function using the formula 1 / (1 + e^{−y_pred}), where y_pred is the predicted value.
* Calculate and plot log-loss values (which can be calculated using the sklearn.metrics.log_loss function) on the training and test samples, and find the minimum metric value and the iteration number at which it is achieved.

In [ ]:
learning_rates = [1, 0.5, 0.3, 0.2, 0.1]

In [ ]:
def sigmoid(y_pred):
    return 1.0 / (1.0 + np.exp(-y_pred))

In [ ]:
# import machine learning algorithms
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

In [ ]:


global_learning_rate = 0
global_iter_num = 0
global_min_log = 1000

for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=250, learning_rate = learning_rate, verbose=True, random_state = 241)
    gb.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test, y_test)))
    # compute test set deviance
    result = []
    iter_num = 0
    min_log = 0
    
    for i, y_pred in enumerate(gb.staged_decision_function(X_test)):
        # clf.loss_ assumes that y_test[i] in {0, 1}
        tmp = log_loss(y_test, sigmoid(y_pred))
        if i == 1:
            min_log = tmp
            iter_num = 1
        if tmp < min_log:
            min_log = tmp
            iter_num = i
        result.append(tmp)
    if global_min_log > min_log:
        global_min_log = min_log
        global_learning_rate = learning_rate
        global_iter_num = iter_num
    plt.plot(result)
    plt.show()
    

In [ ]:
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=250, learning_rate = learning_rate, verbose=True, random_state = 241)
    gb.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test, y_test)))
    # compute test set deviance
    train_loss = []
    test_loss = []
    
    for i, y_pred in enumerate(gb.staged_decision_function(X_test)):
        # clf.loss_ assumes that y_test[i] in {0, 1}
        tmp = log_loss(y_test, sigmoid(y_pred))
        test_loss.append(tmp)
    
    for i, y_pred in enumerate(gb.staged_decision_function(X_train)):
        # clf.loss_ assumes that y_test[i] in {0, 1}
        tmp = log_loss(y_train, sigmoid(y_pred))
        train_loss.append(tmp)
    plt.figure()
    plt.plot(test_loss, 'r', linewidth=2)
    plt.plot(train_loss, 'g', linewidth=2)
    plt.legend(['test', 'train'])
    plt.show()

In [ ]:
print(global_learning_rate, global_iter_num, global_min_log)

How can we characterize the quality graph on the test sample, starting with some iteration: retraining (overfitting) or underfitting (underfitting)? In response, specify one of the words overfitting or underfitting.

In [ ]:
print(1, 'overfitting')

Give the minimum low-loss value on the test sample and the iteration number at which it is reached, with learning_rate = 0.2.

In [ ]:
gb = GradientBoostingClassifier(n_estimators=250, learning_rate = 0.2, verbose=True, random_state = 241)
gb.fit(X_train, y_train)
print("Learning rate: ", learning_rate)
print("Accuracy score (training): {0:.3f}".format(gb.score(X_train, y_train)))
print("Accuracy score (validation): {0:.3f}".format(gb.score(X_test, y_test)))
# compute test set deviance
iter_num = 0
min_log = 0
    
for i, y_pred in enumerate(gb.staged_decision_function(X_test)):
    # clf.loss_ assumes that y_test[i] in {0, 1}
    tmp = log_loss(y_test, sigmoid(y_pred))
    if i == 1:
        min_log = tmp
        iter_num = 1
    if tmp < min_log:
        min_log = tmp
        iter_num = i

In [ ]:
print(iter_num, min_log)

On the same data, train RandomForestClassifier with the number of trees equal to the number of iterations on which the best quality is achieved for gradient boosting from the previous point, random_state=241 and other default parameters. What is the value of logloss on the test obtained from this random forest? (Do not forget that the predictions must be obtained using the predict_proba function. In this case, taking the sigmoid of estimation of probability of class is not necessary)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
clf = RandomForestClassifier(n_estimators=51, random_state=241)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
tmp = log_loss(y_test, y_pred)

In [ ]:
print(tmp)

In [ ]:
clf = RandomForestClassifier(n_estimators=36, random_state=241)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)
tmp = log_loss(y_test, y_pred)

In [ ]:
print(tmp)